In [1]:
import tensorflow as tf
import time
import numpy as np

from tensorflow.keras import datasets, layers, models
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [3]:
LeNet = models.Sequential()
LeNet.add(layers.Conv2D(6, 5, input_shape=(28, 28, 1), padding="same"))
LeNet.add(layers.BatchNormalization())
LeNet.add(layers.ReLU())
LeNet.add(layers.AveragePooling2D(2))

LeNet.add(layers.Conv2D(16, 5, padding="same"))
LeNet.add(layers.BatchNormalization())
LeNet.add(layers.ReLU())
LeNet.add(layers.AveragePooling2D(2))

LeNet.add(layers.Flatten())
LeNet.add(layers.Dense(120))
LeNet.add(layers.ReLU())
LeNet.add(layers.Dense(10))

2022-09-02 14:46:10.258214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 14:46:10.286132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-09-02 14:46:10.286160: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-09-02 14:46:10.287255: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [16]:
LeNet.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 batch_normalization_6 (Batc  (None, 28, 28, 6)        24        
 hNormalization)                                                 
                                                                 
 re_lu_9 (ReLU)              (None, 28, 28, 6)         0         
                                                                 
 average_pooling2d_6 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 16)        2416      
                                                                 
 batch_normalization_7 (Batc  (None, 14, 14, 16)      

In [17]:
LeNet.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
history = LeNet.fit(train_images, train_labels, epochs=10, batch_size=100,
                    validation_data=(test_images, test_labels))

Epoch 1/10
600/600 [==============================] - 14s 24ms/step - loss: 0.1917 - accuracy: 0.9434 - val_loss: 0.0726 - val_accuracy: 0.9757
Epoch 2/10
600/600 [==============================] - 15s 24ms/step - loss: 0.0610 - accuracy: 0.9812 - val_loss: 0.0518 - val_accuracy: 0.9830
Epoch 3/10
600/600 [==============================] - 15s 24ms/step - loss: 0.0470 - accuracy: 0.9855 - val_loss: 0.0382 - val_accuracy: 0.9869
Epoch 4/10
600/600 [==============================] - 15s 25ms/step - loss: 0.0367 - accuracy: 0.9883 - val_loss: 0.0374 - val_accuracy: 0.9879
Epoch 5/10
600/600 [==============================] - 16s 26ms/step - loss: 0.0319 - accuracy: 0.9902 - val_loss: 0.0322 - val_accuracy: 0.9879
Epoch 6/10
600/600 [==============================] - 17s 28ms/step - loss: 0.0271 - accuracy: 0.9918 - val_loss: 0.0331 - val_accuracy: 0.9880
Epoch 7/10
600/600 [==============================] - 16s 26ms/step - loss: 0.0241 - accuracy: 0.9926 - val_loss: 0.0271 - val_accuracy:

In [3]:
def shift_bhw1_into_bhwn(images1c, shifts):
    """Shifts images horizontally and back-fills with zeros.
    @param images: [batch_size, height, width, channels=1]
    @param shifts: [batch_size, n_shifts]
    @output [batch_size, height, width, channels=n_shifts]
    """
    images = tf.tile(images1c, [1, 1, 1, shifts.shape[1]]) # create n_sample_distances channel copies

    left = tf.maximum(0, tf.reduce_max(shifts)) # positive numbers are shifts to the right, for which we need to add zeros on the left
    right = -tf.minimum(0, tf.reduce_min(shifts)) # negative numbers are shifts to the left, for which we need to add zeros on the right
    left_mask = tf.zeros(shape=(tf.shape(images)[0], tf.shape(images)[1], left, tf.shape(images)[3]))
    right_mask = tf.zeros(shape=(tf.shape(images)[0], tf.shape(images)[1], right, tf.shape(images)[3]))
    padded_images = tf.concat([left_mask, images, right_mask], axis=2)

    apply_shifts_to_channels = lambda p: p[0][:, left-p[1]:left-p[1]+images.shape[2]] # p[0] = pair2d, p[1] = shift # positive shift: left-shift
    apply_shifts_for_pair = lambda p: tf.map_fn(apply_shifts_to_channels, (tf.transpose(p[0], perm=[2, 0, 1]), p[1]), dtype=images.dtype) # p[0] = pair3d, p[1] = pair_shifts1d

    result = tf.map_fn(
        apply_shifts_for_pair,
        (padded_images, shifts),
        dtype=images.dtype,
    )
    
    return result

In [7]:
class ASL(tf.keras.layers.Layer):
  def __init__(self, num_outputs,**kwargs):
    super(ASL, self).__init__(name="ASL1", **kwargs)
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.out_shape = input_shape
    self.shifts = self.add_weight("shifts", shape=[int(input_shape[3]), 2], 
    trainable=True, initializer = tf.keras.initializers.RandomUniform(minval=-1.0, maxval=1.0))
    #super(ASL, self).build(input_shape)


  def call(self, inputs):
    input_shape = tf.shape(inputs)
    outputs = tf.transpose(inputs, perm=[3, 1, 2, 0])
    outputs = shift_bhw1_into_bhwn(outputs, self.shifts)
    outputs = tf.transpose(outputs, perm=[3, 1, 2, 0])
    return tf.reshape(outputs, self.compute_output_shape(input_shape))
    #return outputs

  def compute_output_shape(self, input_shape):
    return input_shape

In [8]:
test = models.Sequential()
test.add(ASL(6, input_shape = (12,12,1)))
test.summary(show_trainable=True)

TypeError: Exception encountered when calling layer "ASL1" (type ASL).

in user code:

    File "/tmp/ipykernel_36978/1958716804.py", line 16, in call  *
        outputs = shift_bhw1_into_bhwn(outputs, self.shifts)
    File "/tmp/ipykernel_36978/2928162441.py", line 9, in shift_bhw1_into_bhwn  *
        left = tf.maximum(0, tf.reduce_max(shifts)) # positive numbers are shifts to the right, for which we need to add zeros on the left

    TypeError: Input 'y' of 'Maximum' Op has type float32 that does not match type int32 of argument 'x'.


Call arguments received by layer "ASL1" (type ASL):
  • inputs=tf.Tensor(shape=(None, 12, 12, 1), dtype=float32)

In [5]:
LeASLNet = models.Sequential()
LeASLNet.add(layers.Conv2D(6, 5, input_shape=(28, 28, 1), padding="same"))
LeASLNet.add(layers.BatchNormalization())
LeASLNet.add(layers.ReLU())
LeASLNet.add(layers.AveragePooling2D(2))

LeASLNet.add(layers.Conv2D(6, (1, 1)))
LeASLNet.add(layers.BatchNormalization())
LeASLNet.add(layers.ReLU())
LeASLNet.add(ASL(6))
LeASLNet.add(layers.Conv2D(16, (1, 1)))

#LeASLNet.add(layers.BatchNormalization())
#LeASLNet.add(layers.ReLU())
LeASLNet.add(layers.AveragePooling2D(2))

LeASLNet.add(layers.Flatten())
LeASLNet.add(layers.Dense(120))
LeASLNet.add(layers.ReLU())
LeASLNet.add(layers.Dense(10))

In [6]:
LeASLNet.summary(show_trainable=True)

Model: "sequential_1"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 conv2d (Conv2D)             (None, 28, 28, 6)         156       Y          
                                                                            
 batch_normalization (BatchN  (None, 28, 28, 6)        24        Y          
 ormalization)                                                              
                                                                            
 re_lu (ReLU)                (None, 28, 28, 6)         0         Y          
                                                                            
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         Y          
 ooling2D)                                                                  
                                                                            
 conv2d_1 (Conv2D)           (None, 14, 14, 6)        

In [7]:
LeASLNet.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
LeASLNet.get_layer("ASL1").shifts

<tf.Variable 'ASL1/shifts:0' shape=(6, 2) dtype=float32, numpy=
array([[ 0.41557264,  0.08610559],
       [-0.45353985, -0.40028262],
       [ 0.52977204, -0.6483016 ],
       [-0.5562751 ,  0.99467945],
       [ 0.42810202,  0.73710084],
       [ 0.9362438 ,  0.2985592 ]], dtype=float32)>

In [9]:
history = LeASLNet.fit(train_images, train_labels, epochs=2, batch_size=100,
                    validation_data=(test_images, test_labels))

Epoch 1/2
122/600 [=====>........................] - ETA: 11s - loss: 0.7159 - accuracy: 0.7984

KeyboardInterrupt: 

In [113]:
print(history.history)

{'loss': [0.2238084375858307, 0.08032543212175369], 'accuracy': [0.9340833425521851, 0.9759166836738586], 'val_loss': [0.10203221440315247, 0.05954822525382042], 'val_accuracy': [0.9679999947547913, 0.9812999963760376]}


In [12]:
LeASLNet.get_layer("ASL1").shifts

<tf.Variable 'ASL1/shifts:0' shape=(6, 2) dtype=float32, numpy=
array([[ 0.1771493 ,  0.32908297],
       [-0.3509617 , -0.45802355],
       [-0.9182923 ,  0.07526326],
       [ 0.7410233 ,  0.45693517],
       [ 0.7349565 , -0.2704587 ],
       [ 0.4388702 ,  0.01507449]], dtype=float32)>

In [118]:
LeASLNet.get_layer("asl_43").shifts

<tf.Variable 'asl_43/shifts:0' shape=(6, 2) dtype=float32, numpy=
array([[ 0.47383952,  0.53442764],
       [ 0.50214815,  0.45546627],
       [ 0.35216904,  0.2762704 ],
       [ 0.72204304, -0.98372626],
       [ 0.5621507 , -0.22481632],
       [-0.6469619 ,  0.88005996]], dtype=float32)>